***Step 1: Load and preview the data***

In [1]:
# Import necessary libraries and load the dataset
import pandas as pd

# Load the CSV file
df = pd.read_csv('/mnt/data/hotel_bookings.csv')

# Preview the first 5 rows
df.head()


,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,...,deposit_type,agent,company,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status,reservation_status_date
0,Resort Hotel,0,342,2015,July,27,1,0,0,2,...,No Deposit,NaN,NaN,0.0,Transient,0.0,0.0,0.0,Check-Out,2015-07-01
1,Resort Hotel,0,737,2015,July,27,1,0,0,2,...,No Deposit,NaN,NaN,0.0,Transient,0.0,0.0,0.0,Check-Out,2015-07-01
2,Resort Hotel,0,7,2015,July,27,1,0,1,1,...,No Deposit,NaN,NaN,0.0,Transient,75.0,0.0,0.0,Check-Out,2015-07-02
3,Resort Hotel,0,13,2015,July,27,1,0,1,1,...,No Deposit,304.0,NaN,0.0,Transient,75.0,0.0,0.0,Check-Out,2015-07-02
4,Resort Hotel,0,14,2015,July,27,1,0,2,2,...,No Deposit,240.0,NaN,0.0,Transient,98.0,0.0,1.0,Check-Out,2015-07-03


***Step 2: Basic Exploration & Data Quality Check***

In [2]:
# Check the shape of the dataset
df.shape  # Output: (119390, 32)

# Check for missing values
df.isnull().sum().sort_values(ascending=False)

,0
company,40636
agent,8640
country,464
children,4
booking_changes,1
reservation_status_date,1
customer_type,1
adr,1
required_car_parking_spaces,1
days_in_waiting_list,1


***Step 3: Data Cleaning – Handling Missing Values***

In [4]:
# Clean missing values safely (recommended for future pandas versions)

# Replace missing values
df['agent'] = df['agent'].fillna(0)
df['company'] = df['company'].fillna(0)
df['country'] = df['country'].fillna(df['country'].mode()[0])
df['children'] = df['children'].fillna(0)

# Drop any rows with remaining missing values
df = df.dropna()

# Verify there are no missing values left
print(df.isnull().sum().sum())  # Should print 0


0


***Step 4***

***Step 4.1: Number of bookings per hotel type***

In [5]:
# Count number of bookings for each hotel type
hotel_counts = df['hotel'].value_counts()
print(hotel_counts)

hotel
Resort Hotel    40060
City Hotel       3856
Name: count, dtype: int64


***Step 4.2: Cancellation rate per hotel type***

In [6]:
# Calculate the cancellation rate per hotel type
cancellation_rate = df.groupby('hotel')['is_canceled'].mean()

print(cancellation_rate)

hotel
City Hotel      0.310166
Resort Hotel    0.277634
Name: is_canceled, dtype: float64


***Step 4.3: Average Daily Rate (ADR) per hotel type***

In [7]:
# Calculate the average daily rate (ADR) for each hotel type
adr_per_hotel = df.groupby('hotel')['adr'].mean()

print(adr_per_hotel)

hotel
City Hotel      88.373574
Resort Hotel    94.952930
Name: adr, dtype: float64


***Step 4.4: Top months by number of bookings***

In [8]:
# Convert the 'reservation_status_date' column to datetime if it's not already
df['reservation_status_date'] = pd.to_datetime(df['reservation_status_date'])

# Extract the month from the 'reservation_status_date'
df['month'] = df['reservation_status_date'].dt.month

# Count the number of bookings per month
monthly_bookings = df['month'].value_counts().sort_index()

print(monthly_bookings)

month
1     3424
2     3287
3     3592
4     3518
5     3441
6     3224
7     4740
8     5425
9     4476
10    3430
11    2805
12    2554
Name: count, dtype: int64


***Step 4.5: Customer type distribution***

In [9]:
# Count the number of bookings per customer type
customer_type_counts = df['customer_type'].value_counts()

print(customer_type_counts)

customer_type
Transient          31477
Transient-Party     9869
Contract            2269
Group                301
Name: count, dtype: int64


***Step 4.6: Special requests distribution***

In [10]:
# Count the total number of special requests for each customer type
special_requests_by_customer_type = df.groupby('customer_type')['total_of_special_requests'].sum()

print(special_requests_by_customer_type)

customer_type
Contract            1608.0
Group                180.0
Transient          21476.0
Transient-Party     3161.0
Name: total_of_special_requests, dtype: float64
